## Notebook para testes

In [5]:
import base64

import pandas as pd
import os
import requests
import streamlit as st
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [29]:
def get_df_from_csv(csv_name):
    return pd.read_csv(download_csv_file(csv_name), index_col=0)


def download_csv_file(csv_name):
    req = requests.get(
        f"https://raw.githubusercontent.com/pythonbrasil/dados/main/dados/python-brasil-2020/{csv_name}.csv"
    )
    
    if not os.path.exists('./files'):
        os.makedirs('./files')    
    csv_name = f"./files/{csv_name}.csv"
    url_content = req.content
    with open(csv_name, "wb") as csv_file:
        csv_file.write(url_content)
    csv_file.close()
    return csv_name

In [116]:
def get_event_tickets():
    df = get_df_from_csv("youtube-videos")
    df['Tag'] = df['Video title'].str.split(' ').str[0].replace(" ","")

    df["Watch time (hours)"] = df["Watch time (hours)"].round(decimals=2)

    df["Watch time (days)"] = (df["Watch time (hours)"]/24).round(decimals=2)

    df.loc[df['Video title'] == '[PyBr2020] Trilha PEP0 - 2 -  Quinta-feira - Dia 05/11 #pybr2020', 'Video title'] = '[PyBr2020] Trilha PEP0 - Quinta-feira - Dia 05/11 #pybr2020'

    df.loc[df['Video title'] == '[PyBr2020] Trilha PEP0 - 2 - Quarta-feira  - Dia 04/11 #pybr2020', 'Video title'] = '[PyBr2020] Trilha PEP0 - Quarta-feira - Dia 04/11 #pybr2020'

    df.loc[df['Video title'] == '[PyBr2020] Trilha PEP404 - 2 - Quinta-feira - Dia 05/11 #pybr2020', 'Video title'] = '[PyBr2020] Trilha PEP404 - Quinta-feira - Dia 05/11 #pybr2020'

    df = df[df['Tag'] =='[PyBr2020]'][['Video title','Views','Watch time (hours)','Watch time (days)' ]].copy()

    return df.groupby(['Video title']).sum().reset_index()


In [203]:
sample_df = get_event_tickets()
sample_df['Views'].sum()
sample_df['Watch time (hours)'].sum()
sample_df['Watch time (days)'].sum()

427.90000000000003

In [196]:
total_column = 'Views'
categorical_colum = 'Video title'

percent=False

sample_df = sample_df.sort_values(by=total_column)
values = sample_df[total_column]
values_perc = [round(value / sum(values) * 100, 2) for value in values]
categories = sample_df[categorical_colum]


# title = (
#     f"Valores em Percentual: {categorical_column}"
#     if percent
#     else f"Valores em Quantidade: {categorical_column}"
# )

fig = go.Figure()

if percent:
    fig.add_trace(
        go.Bar(
            x=categories,
            y=values_perc,
            name=f"{categorical_column}",
            orientation="v",
            marker={"color": list(range(-10,20))}
        ),
    )
else:
    fig.add_trace(
        go.Bar(
            x=categories,
            y=values,
            name=f"{categorical_column}",
            orientation="v",
            marker={"color": list(range(-10,20))}
        ),
    )
# # # fig.update_layout(title_text=f"{title}")
fig.show()